In [94]:
import numpy as np
import pandas as pd
import json
import matplotlib.pylab as plt
import matplotlib.image as mpimg
%matplotlib inline

In [111]:
submit_097 = pd.read_csv('hand_detection.csv')

with open('data/test.json') as data_file:    
    test_js = json.load(data_file)
test_js = pd.DataFrame(test_js)

In [32]:
clusters = {
    'a1': [332, 304, 71, 54, 129, 300, 214, 169, 67, 321, 28],
    'a2': [184, 335, 166, 104, 249, 209, 22, 3, 49, 200],
    'a3': [113, 167, 225, 236, 253, 135, 115, 261, 123, 152],
    'a4': [17, 310, 206],
    'b1': [243, 163, 26, 199, 256, 319],
    'b2': [196, 14, 154, 153, 13, 313, 238, 193, 84, 230, 65, 101, 18, 280],
    'b3': [38, 179, 287, 32, 283, 156, 242, 244, 317, 161],
    'b4': [96, 269, 322, 151, 212, 60, 202, 189],
    'b5': [308, 146, 316, 150, 47, 259, 155],
    'c1': [33, 148, 134, 258, 325, 56, 37, 46],
    'c2': [126, 187, 326, 140, 260, 174, 6, 133, 127],
    'c3': [125, 226, 122, 120, 254],
    'c4': [284, 231, 88, 208, 333, 48, 233, 138, 172, 23],
    'c5': [164, 331, 1, 85, 90, 324, 291, 111, 52],
    'c6': [82, 168, 305, 299, 19, 271, 86],
    'd1': [221, 136, 302, 159],
    'd2': [275, 147, 63, 216, 72, 227, 219, 149, 114, 81, 9],
    'd3': [341, 27, 198, 328],
    'd4': [87, 330, 215, 100, 128, 24, 323, 102, 186, 142, 337],
    'd5': [139, 293, 130, 278, 119, 191, 183, 2, 320],
    'd6': [182, 294, 66, 124],
    'e1': [232, 276, 197, 314, 8, 343, 336, 247, 39, 70, 240, 273, 228, 315, 344],
    'e2': [192, 290, 7, 265, 106, 132, 338, 51, 89, 98, 282, 292, 295, 141, 105, 270, 266],
    'e3': [117, 250, 95, 80, 93, 339, 246, 210, 286, 92, 40, 34, 110, 99, 180, 73, 234, 248, 263, 108, 15, 195],
    'e4': [11, 181, 185, 272, 91, 188, 64, 30, 75, 145, 235, 77, 207, 204, 222, 306],
    'e5': [41, 177, 281, 42, 83, 245, 121, 157, 296, 334, 170]
}

In [95]:
def print_set(folder, set_n, order=[1, 2, 3, 4, 5]):
    fig, ax = plt.subplots(1, 5, sharey=True, figsize=(20, 4))
    ax[0].set_ylabel('Set: ' + str(set_n), fontsize=30)
    ax_cnt = 0
    for i in order:
        img_data = mpimg.imread('data/' + folder + '/set' + str(set_n) + '_' + str(i) + '.jpeg')
        ax[ax_cnt].imshow(img_data)
        ax[ax_cnt].set_title('Image: ' + str(i), fontsize=18)
        ax[ax_cnt].set_xticks([])
        ax[ax_cnt].set_yticks([])
        ax_cnt += 1

    fig.tight_layout()

In [151]:
from sklearn.metrics import mean_squared_error as mse

def get_order(order):
    return [int(x) for x in order.split(' ')]

def get_angles(set_n, order):
    angles = []
    set_name = 'set' + str(set_n) + '_'
    for i in range(len(order) - 1):
        angles.append(
            test_js[test_js['name'] == set_name]['data'].values[0][str(order[i])][str(order[i + 1])]['angle']
        )
    return angles

def get_scales(set_n, order):
    angles = []
    set_name = 'set' + str(set_n) + '_'
    for i in range(len(order) - 1):
        angles.append(
            test_js[test_js['name'] == set_name]['data'].values[0][str(order[i])][str(order[i + 1])]['scale']
        )
    return angles

def print_error(data, cluster, right_set, feature='angle'):
    order_right = get_order(data[data['setId'] == right_set]['day'].values[0])
    feature_right = get_angles(right_set, order_right) if feature == 'angle' \
        else get_scales(right_set, order_right)
    for s in cluster:
        if(s != right_set):
            order_s = get_order(data[data['setId'] == s]['day'].values[0])
            feature_s = get_angles(s, order_s) if feature == 'angle' \
                else get_scales(s, order_s)
            print ('%d:\t%f') % (s, mse(feature_s, feature_right))
            
def print_error_first(data, cluster, feature='angle'):
    print_error(data, cluster, cluster[0], feature)

In [149]:
print_error(submit_097, clusters['b1'], 199)

243:	0.472057
163:	0.003889
26:	0.218223
256:	0.308654
319:	0.228850


In [150]:
print_error(submit_097, clusters['b1'], 199, feature='scale')

243:	0.000004
163:	0.000006
26:	0.000019
256:	0.000006
319:	0.000005


In [152]:
for cl in sorted(clusters.keys()):
    print 'Cluster:\t' + str(cl)
    print_error_first(submit_097, clusters[cl])
    print

Cluster:	a1
304:	2.821662
71:	3.678097
54:	0.109521
129:	7.065501
300:	6.755340
214:	1.988129
169:	0.551259
67:	0.740562
321:	0.473963
28:	4.078316

Cluster:	a2
335:	0.174074
166:	0.103148
104:	1.234259
249:	82.598930
209:	91.501071
22:	88.582814
3:	87.793851
49:	85.208100
200:	83.163453

Cluster:	a3
167:	0.455011
225:	0.297108
236:	2.816470
253:	9.624140
135:	8.791579
115:	11.818499
261:	14.088787
123:	12.011482
152:	11.460736

Cluster:	a4
310:	0.036254
206:	0.012617

Cluster:	b1
163:	0.543974
26:	1.256917
199:	0.472057
256:	0.035118
319:	1.263101

Cluster:	b2
14:	0.594948
154:	0.140468
153:	0.010789
13:	0.421159
313:	0.200217
238:	5.901167
193:	5.778784
84:	4.702701
230:	5.886526
65:	3.034461
101:	5.840063
18:	7.535821
280:	7.371717

Cluster:	b3
179:	0.140253
287:	9769.946634
32:	2.290755
283:	0.333209
156:	1.835924
242:	0.382924
244:	1.539473
317:	1.131073
161:	1.145893

Cluster:	b4
269:	0.062225
322:	43.778734
151:	4.139509
212:	0.896366
60:	2.793102
202:	1.100775
189:	3.017738

Cl

In [153]:
for cl in sorted(clusters.keys()):
    print 'Cluster:\t' + str(cl)
    print_error_first(submit_097, clusters[cl], feature='scale')
    print

Cluster:	a1
304:	0.000012
71:	0.000031
54:	0.000007
129:	0.000019
300:	0.000032
214:	0.000053
169:	0.000025
67:	0.000054
321:	0.000007
28:	0.000053

Cluster:	a2
335:	0.000014
166:	0.000061
104:	0.000101
249:	0.000670
209:	0.000446
22:	0.000796
3:	0.000376
49:	0.000672
200:	0.000506

Cluster:	a3
167:	0.000022
225:	0.000160
236:	0.000013
253:	0.000134
135:	0.000009
115:	0.000165
261:	0.000093
123:	0.000074
152:	0.000046

Cluster:	a4
310:	0.000003
206:	0.000009

Cluster:	b1
163:	0.000003
26:	0.000019
199:	0.000004
256:	0.000012
319:	0.000011

Cluster:	b2
14:	0.000010
154:	0.000005
153:	0.000004
13:	0.000008
313:	0.000007
238:	0.000005
193:	0.000006
84:	0.000003
230:	0.000008
65:	0.000015
101:	0.000008
18:	0.000004
280:	0.000040

Cluster:	b3
179:	0.000081
287:	3.812144
32:	0.000048
283:	0.000058
156:	0.000006
242:	0.000068
244:	0.000055
317:	0.000031
161:	0.000123

Cluster:	b4
269:	0.000229
322:	0.000057
151:	0.000026
212:	0.000022
60:	0.000032
202:	0.000069
189:	0.000057

Cluster:	b5
146: